In [ ]:
import nltk
import pandas as pd
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import time

# Download the VADER lexicon if not already downloaded
nltk.download('vader_lexicon')

# Record the start time
start_time = time.time()

# Load your dataset with the appropriate encoding (e.g., UTF-8)
data = pd.read_csv('tdataset.csv', encoding='utf-8')

# Initialize VADER sentiment analyzer
sid = SentimentIntensityAnalyzer()

# Function to analyze sentiment using VADER with modified threshold for neutrality
def analyze_sentiment(text):
    # Handle float values by converting them to strings
    text = str(text)
    sentiment_score = sid.polarity_scores(text)

    # Adjust the threshold for neutrality (closer to 0) for classifying as neutral
    if -0.05 < sentiment_score['compound'] < 0.05:
        return 'Neutral'
    elif sentiment_score['compound'] >= 0.05:
        return 'Positive'
    else:
        return 'Negative'

# Apply sentiment analysis to the 6th column (0-based index) and create a new column 'sentiment'
data['sentiment'] = data.iloc[:, 5].apply(analyze_sentiment)

# Save the original data with sentiment analysis results to a different CSV file
data.to_csv('sentiment_analysis_results_output.csv', index=False, encoding='utf-8')

# Record the end time
end_time = time.time()

# Calculate and display the execution time
execution_time = end_time - start_time
print(f"Execution Time: {execution_time} seconds")

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


Execution Time: 37.831063747406006 seconds


In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import time

# Initialize a Spark session
spark = SparkSession.builder.appName("SentimentAnalysis").getOrCreate()
sc = spark.sparkContext

# Record the start time
start_time = time.time()

# Load your dataset into a distributed DataFrame
data = spark.read.csv('tdataset.csv', header=True, inferSchema=True, encoding='ISO-8859-1')

# Rename the 6th column (0-based index) to a simpler name, e.g., "text"
data = data.withColumnRenamed(data.columns[5], "text")

# Initialize VADER sentiment analyzer
sid = SentimentIntensityAnalyzer()

# Function to analyze sentiment using VADER with modified threshold for neutrality
def analyze_sentiment(text):
    text = str(text)
    sentiment_score = sid.polarity_scores(text)

    if -0.05 < sentiment_score['compound'] < 0.05:
        return 'Neutral'
    elif sentiment_score['compound'] >= 0.05:
        return 'Positive'
    else:
        return 'Negative'

# Create a user-defined function (UDF) to apply the sentiment analysis function
analyze_sentiment_udf = udf(analyze_sentiment, StringType())

# Apply sentiment analysis to the "text" column using the UDF
data = data.withColumn("sentiment", analyze_sentiment_udf(data["text"]))

# Save the "text" and "sentiment" columns to a CSV file
data.select("text", "sentiment").coalesce(1).write.csv('sentiment_analysis_results.csv', header=True, mode="overwrite")

# Record the end time
end_time = time.time()

# Calculate and display the execution time
execution_time = end_time - start_time
print(f"Execution Time: {execution_time} seconds")

# Stop the Spark session
spark.stop()

Execution Time: 39.85838508605957 seconds


In [ ]:
import nltk
import pandas as pd
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import time

# Download the VADER lexicon if not already downloaded
nltk.download('vader_lexicon')

# Record the start time
start_time = time.time()

# Load your dataset with the appropriate encoding (e.g., UTF-8)
data = pd.read_csv('tdataset.csv', encoding='utf-8')

# Initialize VADER sentiment analyzer
sid = SentimentIntensityAnalyzer()

# Create an empty list to store sentiment analysis results
sentiment_results = []

# Function to analyze sentiment using VADER with modified threshold for neutrality
def analyze_sentiment(text):
    # Handle float values by converting them to strings
    text = str(text)
    sentiment_score = sid.polarity_scores(text)

    # Adjust the threshold for neutrality (closer to 0) for classifying as neutral
    if -0.05 < sentiment_score['compound'] < 0.05:
        return 'Neutral'
    elif sentiment_score['compound'] >= 0.05:
        return 'Positive'
    else:
        return 'Negative'

# Iterate through the DataFrame and perform sentiment analysis for each row
for index, row in data.iterrows():
    sentiment_result = analyze_sentiment(row[5])  # Assuming the text is in the 6th column (0-based index)
    sentiment_results.append(sentiment_result)

# Add the sentiment analysis results as a new column in the DataFrame
data['sentiment'] = sentiment_results

# Save the original data with sentiment analysis results to a different CSV file
data.to_csv('sentiment_analysis_results_output.csv', index=False, encoding='utf-8')

# Record the end time
end_time = time.time()

# Calculate and display the execution time
execution_time = end_time - start_time
print(f"Execution Time: {execution_time} seconds")

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


Execution Time: 50.49564719200134 seconds
